## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp

## 1. t-Test 기반 모델1,2 구분성 상위 특징 인덱스 가져오기

In [2]:
Rank = 30

MC_P_value_Rank_M1 = pd.read_csv('./ProcessedData/MC_P_value_Rank_M1', sep = ',', header=None) # Model 1 : 정상 / 고장 간 구분성상위 특징
MC_P_value_Rank_M2 = pd.read_csv('./ProcessedData/MC_P_value_Rank_M2', sep = ',', header=None) # Model 2 : 고장 1 / 2 간 구분성상위 특징

# 구분성 상위 일정 개수(Rank) 특징의 인덱스 추출
M1_FeatureIndex = np.array(MC_P_value_Rank_M1.iloc[:,0], dtype = int)[:Rank] # Model 1
M2_FeatureIndex = np.array(MC_P_value_Rank_M2.iloc[:,0], dtype = int)[:Rank] # Model 2
M1_FeatureIndex

array([180, 170,  83,  81, 178, 189, 261, 187, 184, 183,  73,  91, 174,
        93, 173,  97, 101,  84, 179,  90, 177,  28, 108, 103, 160,  99,
        94,  74,  13, 176])

## 2. 모델 별 데이터 가공 및 분할, 레이블 생성 : 학습, 검증 각각 150, 30개씩

In [3]:
# 추출한 특징 불러오기
MC_FeatureData = pd.read_csv('./ProcessedData/MC_FeatureData', sep=',', header=None)

# 특징 데이터 정상/고장1/고장2 분리
NoOfData = int(MC_FeatureData.shape[1]/3)
Normal_FeatureData    = MC_FeatureData.iloc[ : ,            :   NoOfData ]
Abnormal1_FeatureData = MC_FeatureData.iloc[ : ,   NoOfData : 2*NoOfData ]
Abnormal2_FeatureData = MC_FeatureData.iloc[ : , 2*NoOfData :            ]

print(Normal_FeatureData.shape, Abnormal1_FeatureData.shape, Abnormal2_FeatureData.shape)#kfold교차검증 해보는 것도 도움이 될 수 있다.

(270, 180) (270, 180) (270, 180)


In [5]:
print(NoOfData)

180


In [4]:
NoOfTrainData = 150
NoOfValidData = NoOfData - NoOfTrainData
# 학습, 검증용 데이터 분할
Train_Normal_FeatureData    = Normal_FeatureData.iloc[:,:NoOfTrainData].T
Train_Abnormal1_FeatureData = Abnormal1_FeatureData.iloc[:,:NoOfTrainData].T
Train_Abnormal2_FeatureData = Abnormal1_FeatureData.iloc[:,:NoOfTrainData].T # 학습
Valid_Normal_FeatureData    = Normal_FeatureData.iloc[:,NoOfTrainData:].T
Valid_Abnormal1_FeatureData = Abnormal1_FeatureData.iloc[:,NoOfTrainData:].T
Valid_Abnormal2_FeatureData = Abnormal1_FeatureData.iloc[:,NoOfTrainData:].T # 검증

M1_Train_AllFeature = pd.concat([Train_Normal_FeatureData,Train_Abnormal1_FeatureData,Train_Abnormal2_FeatureData], axis = 0)
M2_Train_AllFeature = pd.concat([Train_Abnormal1_FeatureData,Train_Abnormal2_FeatureData], axis = 0)
Valid_AllFeature = pd.concat([Valid_Normal_FeatureData,Valid_Abnormal1_FeatureData,Valid_Abnormal2_FeatureData], axis = 0)

print('Model 1 Train shape(270개 특징 중 {}개 사용):'.format(Rank),M1_Train_AllFeature.shape)
print('Model 2 Train shape(270개 특징 중 {}개 사용):'.format(Rank),M2_Train_AllFeature.shape)
print('Model 1, 2 학습/검증에 사용되는 특징 다름 주의!')

Model 1 Train shape(270개 특징 중 30개 사용): (450, 270)
Model 2 Train shape(270개 특징 중 30개 사용): (300, 270)
Model 1, 2 학습/검증에 사용되는 특징 다름 주의!


In [6]:
### 레이블 만들기
## 학습
# KNN, SVM model 1 : 정상 / 고장1 : 0 / 1
M1_Train_Label = np.zeros(3*NoOfTrainData , dtype=int)
M1_Train_Label[NoOfTrainData:] = 1
# ANN model 1: 정상 / 고장 [1,0] / [0,1]
M1_Train_Label_forANN = np.zeros((3*NoOfTrainData, 2) , dtype=int)
M1_Train_Label_forANN[:NoOfTrainData, :] = [1,0]
M1_Train_Label_forANN[NoOfTrainData:, :] = [0,1]

# KNN, SVM model 2 : 고장1 / 고장2 : 1 / 2
M2_Train_Label = np.ones(2*NoOfTrainData , dtype=int)
M2_Train_Label[NoOfTrainData:] = 2
# ANN model 2: 고장1 / 고장2 [1,0] / [0,1]
M2_Train_Label_forANN = np.zeros((2*NoOfTrainData, 2) , dtype=int)
M2_Train_Label_forANN[:NoOfTrainData, :] = [1,0]
M2_Train_Label_forANN[NoOfTrainData:, :] = [0,1]

## 검증 레이블
Valid_Label = np.zeros(3*NoOfValidData , dtype=int)
Valid_Label[  NoOfValidData:2*NoOfValidData] = 1
Valid_Label[2*NoOfValidData:               ] = 2

print('Model 1 Training label:', M1_Train_Label.shape, M1_Train_Label_forANN.shape)
print('Model 2 Training label:', M2_Train_Label.shape, M2_Train_Label_forANN.shape)
print('Validation label:', Valid_Label.shape)

Model 1 Training label: (450,) (450, 2)
Model 2 Training label: (300,) (300, 2)
Validation label: (90,)


## 3. 모델 1,2 학습/검증

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras

### 3-1. KNN

In [ ]:
# 학습: 모델 별 학습 데이터 중 구분성 상위 특징 가져오기
M1_Train = M1_Train_AllFeature.iloc[:, M1_FeatureIndex]
M2_Train = M2_Train_AllFeature.iloc[:, M2_FeatureIndex]

knn_Model1 = KNeighborsClassifier(n_neighbors=7).fit(M1_Train, M1_Train_Label) # 정상/고장 구분 모델
knn_Model2 = KNeighborsClassifier(n_neighbors=9).fit(M2_Train, M2_Train_Label) # 고장1/고장2 구분 모델

In [ ]:
## 검증 정확도 확인
M1_Valid = np.array(Valid_AllFeature)[:, M1_FeatureIndex] # 검증 데이터 특징 중 정상/고장 구분성 높은 것 가져오기
M1_predict = knn_Model1.predict(M1_Valid)                 # Model 1 진단 결과
M1_predict_AbnormalIndex = np.where(M1_predict==1)[0]     # 고장 Label : 1 고장인 애들만 찾아서

M2_Valid = np.array(Valid_AllFeature)[:, M2_FeatureIndex] # 검증 데이터 특징 중 고장 1/2 간 구분성 높은 것 가져오기
M2_Valid = M2_Valid[M1_predict_AbnormalIndex, :]          # 데이터 중 M1에서 고장이라 진단한 것들만 가져오기
M2_predict = knn_Model2.predict(M2_Valid)                 # Model 2 진단 결과

# Model 1, 2 결과 합치기
FinalPredict = np.copy(M1_predict)
for i in range(len(M1_predict_AbnormalIndex)):
    if M2_predict[i] == 2:
        FinalPredict[M1_predict_AbnormalIndex[i]] = 2
        
Accuracy = metrics.accuracy_score(Valid_Label, FinalPredict)
print('kNN+kNN model Accuracy: {:.2f}%'.format(Accuracy*100))

### 3-2. SVM

In [ ]:
# 학습: 모델 별 학습 데이터 중 구분성 상위 특징 가져오기
M1_Train = M1_Train_AllFeature.iloc[:, M1_FeatureIndex]
M2_Train = M2_Train_AllFeature.iloc[:, M2_FeatureIndex]

svm_Model1 = SVC(kernel = 'poly').fit(M1_Train, M1_Train_Label) # 정상/고장 구분 모델
svm_Model2 = SVC(kernel = 'poly').fit(M2_Train, M2_Train_Label) # 고장1/고장2 구분 모델

In [ ]:
## 검증 정확도 확인
M1_Valid = np.array(Valid_AllFeature)[:, M1_FeatureIndex] # 검증 데이터 특징 중 정상/고장 구분성 높은 것 가져오기
M1_predict = svm_Model1.predict(M1_Valid)                 # Model 1 진단 결과
M1_predict_AbnormalIndex = np.where(M1_predict==1)[0]     # 고장 Label : 1

M2_Valid = np.array(Valid_AllFeature)[:, M2_FeatureIndex] # 검증 데이터 특징 중 고장 1/2 간 구분성 높은 것 가져오기
M2_Valid = M2_Valid[M1_predict_AbnormalIndex, :]          # 데이터 중 M1에서 고장이라 진단한 것들만 가져오기
M2_predict = svm_Model2.predict(M2_Valid)                 # Model 2 진단 결과

# Model 1, 2 결과 합치기
FinalPredict = np.copy(M1_predict)
for i in range(len(M1_predict_AbnormalIndex)):
    if M2_predict[i] == 2:
        FinalPredict[M1_predict_AbnormalIndex[i]] = 2
        
Accuracy = metrics.accuracy_score(Valid_Label, FinalPredict)
print('SVM+SVM model Accuracy: {:.2f}%'.format(Accuracy*100))

### ANN

In [ ]:
learningRate  = 0.0001
noOfNeuron    = 16
iteration     = 1000

def ANN_model(input_data):
    model = keras.Sequential()
    model.add(keras.layers.Dense(units = noOfNeuron, input_shape = (input_data.shape[1],) ))  # Input  Layer
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu))    # Hidden Layer 1
    model.add(keras.layers.Dense(units = noOfNeuron, activation = keras.activations.relu))    # Hidden Layer 2
    model.add(keras.layers.Dense(units = 2,          activation = keras.activations.softmax)) # Output Layer
    
    model.compile(optimizer= keras.optimizers.Adam(learning_rate = learningRate),
                  loss=keras.losses.categorical_crossentropy,
                  metrics=['accuracy'])
    return model

In [ ]:
# 모델 따로 생성
ann_model1 = ANN_model(M1_Train)
ann_model2 = ANN_model(M2_Train)

In [ ]:
# 학습: 모델 별 학습 데이터 중 구분성 상위 특징 가져오기
M1_Train = M1_Train_AllFeature.iloc[:, M1_FeatureIndex]
M2_Train = M2_Train_AllFeature.iloc[:, M2_FeatureIndex]

# Model 1
hist1 = ann_model1.fit(M1_Train, M1_Train_Label_forANN, epochs=iteration, verbose = 0) # 정상/고장 구분 모델
print('Model 1 Training Done')
# Model 2
hist2 = ann_model2.fit(M2_Train, M2_Train_Label_forANN, epochs=iteration, verbose = 0) # 고장1/고장2 구분 모델
print('Model 2 Training Done')

In [ ]:
## 검증 정확도 확인
M1_Valid = np.array(Valid_AllFeature)[:, M1_FeatureIndex]            # 검증 데이터 특징 중 정상/고장 구분성 높은 것 가져오기
M1_predict = (ann_model1.predict(M1_Valid) > 0.5).astype("int32")    # Model 1 진단 결과
M1_predict_AbnormalIndex = np.unique(np.where(M1_predict==[0,1])[0]) # 고장 Label : [0,1]

M2_Valid = np.array(Valid_AllFeature)[:, M2_FeatureIndex]            # 검증 데이터 특징 중 고장 1/2 간 구분성 높은 것 가져오기
M2_Valid = M2_Valid[M1_predict_AbnormalIndex, :]                     # 데이터 중 M1에서 고장이라 진단한 것들만 가져오기
M2_predict = (ann_model1.predict(M2_Valid) > 0.5).astype("int32")    # Model 2 진단 결과

# Model 1, 2 진단 결과 합치기
M1_predict = pd.DataFrame(M1_predict)
M2_predict = pd.DataFrame(M2_predict, index = M1_predict_AbnormalIndex)
FinalPredict_1hot = pd.concat([M1_predict,M2_predict], axis = 1).fillna(0)
FinalPredict = np.zeros(M1_Valid.shape[0], dtype = int)
for i in range(M1_Valid.shape[0]):
    if np.all(FinalPredict_1hot.iloc[i,:] == [1,0,0,0]):
        FinalPredict[i] = 0
    elif np.all(FinalPredict_1hot.iloc[i,:] == [0,1,1,0]):
        FinalPredict[i] = 1
    else:
        FinalPredict[i] = 2
        
Accuracy = metrics.accuracy_score(Valid_Label, FinalPredict)
print('ANN+ANN model Accuracy: {:.2f}%'.format(Accuracy*100))